In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt


def log2(p):
    return np.ma.log2(p).filled(0)


def dataframe_entropy(d: pd.DataFrame, column: str):
    return entropy_singular(d[column].value_counts()[0] / len(d))


def entropy(p):
    ap = 1-p
    return -(p*log2(p) + ap*log2(ap))


def entropy_singular(p):
    if p == 0 or p == 1:
        return 0
    ap = 1-p
    return -(p*np.log2(p) + ap*np.log2(ap))


raw_dataset = pd.read_csv("./dogscats-categorical.csv")
raw_dataset.tail()


,Ears shape,Face shape,Whiskers,Cat
5,Pointy,Round,Absent,Yes
6,Floppy,Not round,Absent,No
7,Pointy,Round,Absent,Yes
8,Floppy,Round,Absent,No
9,Floppy,Round,Absent,No


In [2]:
dataset = raw_dataset.copy()
dataset = pd.get_dummies(dataset, columns=['Ears shape'], prefix='', prefix_sep='')
dataset.tail()


,Face shape,Whiskers,Cat,Floppy,Oval,Pointy
5,Round,Absent,Yes,False,False,True
6,Not round,Absent,No,True,False,False
7,Round,Absent,Yes,False,False,True
8,Round,Absent,No,True,False,False
9,Round,Absent,No,True,False,False


In [3]:
import pprint


def dt_split(ds: pd.DataFrame, by: str, ignore: list[str] = [], verbose=False):
    uniques = ds[by].unique()
    ruler = uniques[0]
    BASE_ENTROPY = dataframe_entropy(ds, by)
    if verbose:
        print(f"Unique values in {by}: {uniques}")
        print(f"BASE ENTROPY: {BASE_ENTROPY}")
    SPLIT_ENTROPIES_VERBOSE = {}
    SPLIT_ENTROPIES = []
    for col in ds.columns:
        if col in ignore or col == by:
            continue
        splits = list(ds.groupby(col))
        entr1 = entr2 = avg = 0
        (dgr1n, dgr1) = splits[0]
        (dgr2n, dgr2) = splits[1] if len(splits) > 1 else ('UNKNOWN', None)
        entr1 = dataframe_entropy(dgr1, by)
        avg = entr1 * len(dgr1) / len(ds)
        if dgr2 is not None:
            entr2 = dataframe_entropy(dgr2, by)
            avg += entr2 * len(dgr2) / len(ds)
        reduction = BASE_ENTROPY - avg
        SPLIT_ENTROPIES.append([col, reduction])
        SPLIT_ENTROPIES_VERBOSE[col] = (((dgr1n, entr1, dgr1[by].iloc[0]),
                                        (dgr2n, entr2, None if dgr2 is None else dgr2[by].iloc[0])),
                                        avg)
        if verbose:
            print("--------------------------")
            print(f"{col}:")
            vc = dgr1[by].value_counts()
            print(f"{vc[ruler] if ruler in vc else 0}/{len(dgr1)} {dgr1n} --> E {entr1}")
            if dgr2 is not None:
                vc = dgr2[by].value_counts()
                print(f"{vc[ruler] if ruler in vc else 0}/{len(dgr2)} {dgr2n} --> E {entr2}")
            print(f"A {avg}")
            print(f"R {reduction}")
            print("--------------------------")
    SPLIT_ENTROPIES = np.array(SPLIT_ENTROPIES)
    winner = SPLIT_ENTROPIES[np.argmax(SPLIT_ENTROPIES[:, 1])][0]
    sufficient = SPLIT_ENTROPIES_VERBOSE[winner][1] == 0
    return winner, sufficient, SPLIT_ENTROPIES_VERBOSE


def recursive_split(ds: pd.DataFrame, by: str, attach_to: dict | None = None, ignore: list[str] = [], verbose=False, depth=0):
    if attach_to is None:
        attach_to = {}
    w, ws, wv = dt_split(ds=ds, by=by, ignore=ignore, verbose=verbose)
    print(f"[{depth}]|{ignore}||||||||{w}: {ws}")
    (n1, e1, v1), (n2, e2, v2) = wv[w][0]
    print(f"[{depth}]|{ignore} ENTR {e1}/{e2} OPTS {n1}/{n2}")
    if ws:
        attach_to[w] = {n1: v1, n2: v2}
    else:
        attach_to[w] = {}
        s1, s2 = ds.groupby(w)
        print(f"[{depth}]|{ignore}Attaching <{n1}>{e1} & <{n2}>{e2} TO {w}:")
        attach_to[w][n1] = {}
        attach_to[w][n2] = {}
        print(f"[{depth}]|{ignore}{attach_to}")
        print(f"[{depth}]|{ignore}Attaching s1:")
        if e1 > 0:
            recursive_split(ds=s1[1], by=by, attach_to=attach_to[w][n1], ignore=[*ignore, w], verbose=verbose, depth=depth+1)
        else:
            attach_to[w][n1] = v1
        print(f"[{depth}]|{ignore}{attach_to}")
        print(f"[{depth}]|{ignore}Attaching s2:")
        if e2 > 0:
            recursive_split(ds=s2[1], by=by, attach_to=attach_to[w][n2], ignore=[*ignore, w], verbose=verbose, depth=depth+1)
        else:
            attach_to[w][n2] = v2
        print(f"[{depth}]|{ignore}{attach_to}")
        print(f"[{depth}]|{ignore}Attaching DONE")
    return attach_to

tree = recursive_split(dataset, "Cat", verbose=False)
pprint.pprint(tree)


[0]|[]||||||||Floppy: False
[0]|[] ENTR 0.863120568566631/0 OPTS False/True
[0]|[]Attaching <False>0.863120568566631 & <True>0 TO Floppy:
[0]|[]{'Floppy': {False: {}, True: {}}}
[0]|[]Attaching s1:
[1]|['Floppy']||||||||Face shape: False
[1]|['Floppy'] ENTR 1.0/0.7219280948873623 OPTS Not round/Round
[1]|['Floppy']Attaching <Not round>1.0 & <Round>0.7219280948873623 TO Face shape:
[1]|['Floppy']{'Face shape': {'Not round': {}, 'Round': {}}}
[1]|['Floppy']Attaching s1:
[2]|['Floppy', 'Face shape']||||||||Oval: True
[2]|['Floppy', 'Face shape'] ENTR 0/0 OPTS False/True
[1]|['Floppy']{'Face shape': {'Not round': {'Oval': {False: 'No', True: 'Yes'}}, 'Round': {}}}
[1]|['Floppy']Attaching s2:
[2]|['Floppy', 'Face shape']||||||||Oval: False
[2]|['Floppy', 'Face shape'] ENTR 0/1.0 OPTS False/True
[2]|['Floppy', 'Face shape']Attaching <False>0 & <True>1.0 TO Oval:
[2]|['Floppy', 'Face shape']{'Oval': {False: {}, True: {}}}
[2]|['Floppy', 'Face shape']Attaching s1:
[2]|['Floppy', 'Face shape']{